# BP Experiments to verify theory for L-0 norm - CIFAR-10

In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division
from __future__ import print_function

import sys, os, gc, math
import numpy as np
from scipy.fftpack import dct,idct
from keras.datasets import mnist, fashion_mnist,cifar10
from PIL import Image
from multiprocessing import Pool
import time


sys.path.append('../')

from models.util import *


#Seed used for choosing classes, training points, and test points.
#SEED = 14
SEED=11

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
num_samples = 500
sqrt_n = 32
input_shape=(sqrt_n,sqrt_n,1)
n = sqrt_n*sqrt_n
k = 10
c=2.0

In [3]:
def four(k,n,c):
    a = ((n/c) - (k**2))**2
    b = (c/(n*k))
    return a*(1/4.0)*b

def book(k,n,c):
    return (np.sqrt(n/(c*41))*4 - 2*k  )*0.5 

def rns(k,t,n,c):
    return ( np.sqrt(k+t)*(1.0/np.sqrt(k) + 1.0/np.sqrt(t))*np.sqrt((4*c*k*t)/n))

In [4]:
#Load Cifar10 data
(X_train, _), (X_test, _) = cifar10.load_data()
m_data = np.concatenate((X_train,X_test))

In [5]:
#Normalize the data
m_data = m_data/255.0

In [6]:
#Check Cifar10 results for 500 random images - BP
t = int(four(float(k),float(n),float(c)))
print('Noise budget of %d' % t)
subset_idx = np.random.choice(np.arange(m_data.shape[0]),num_samples)
m_data_sub_bp = m_data[subset_idx]
m_data_y_bp = np.zeros((num_samples,sqrt_n,sqrt_n,3))
t_values_bp = np.zeros((num_samples,3))
for i in range(num_samples):
    #first sample an element from the data
    x_r = m_data_sub_bp[i,:,:,0].flatten()
    x_g = m_data_sub_bp[i,:,:,1].flatten()
    x_b = m_data_sub_bp[i,:,:,2].flatten()

    #Now sample a t - must be atleast 1
    t_r = np.random.randint(1,t)
    t_g = np.random.randint(1,t)
    t_b = np.random.randint(1,t)
    t_values_bp[i,0] = t_r
    t_values_bp[i,1] = t_g
    t_values_bp[i,2] = t_b


    #Now samnple the an index set
    s_r = np.random.choice(np.arange(n),t_r)
    s_g = np.random.choice(np.arange(n),t_g)
    s_b = np.random.choice(np.arange(n),t_b)


    e = np.zeros((n,3))
    #Now create the vector e
    #pick a value for each element between 0 and 1 as the images are normalized
    for j in range(t_r):
        e[s_r[j],0] = np.random.uniform()
    for j in range(t_g):
        e[s_g[j],1] = np.random.uniform()
    for j in range(t_b):
        e[s_b[j],2] = np.random.uniform()
    y = m_data_sub_bp[i,:,:,:] + e.reshape((sqrt_n,sqrt_n,3))
    m_data_y_bp[i,:,:,:] = y

Noise budget of 8


# Get \delta_p and \Delta_p

In [7]:
#Form the matrix F
F = get_matrix(n,tf='dct')
I = np.identity(n)
A = np.concatenate((F.T,I), axis=1)
n = 32*32

In [8]:
def approximate(z):
    y,eta = z
    x_hat = socp(y,A,n=2*n,eta=eta)[:n]
    return x_hat

In [ ]:
%%capture three
errors_l2_bp = np.zeros(m_data_y_bp.shape[0])
bot_l2_bp = np.zeros((m_data_y_bp.shape[0],3))
diff_l2_bp = np.zeros(m_data_y_bp.shape[0])

for i in range(num_samples):
    y_r = m_data_y_bp[i,:,:,0].flatten()
    x_r = m_data_sub_bp[i,:,:,0].flatten()
    
    #Get actual top k and bottom k (we use the faster transform which may introduce some error)
    x_hat_top_k_r, x_hat_bot_k_r =  get_top_bot_k_vec(dct(x_r, norm='ortho'),k=k)    
    e_r = y_r - x_r 
    eta_r = bot_l2_bp[i,0] = np.linalg.norm(x_hat_bot_k_r)
    
    y_g = m_data_y_bp[i,:,:,1].flatten()
    x_g = m_data_sub_bp[i,:,:,1].flatten()
    
    #Get actual top k and bottom k (we use the faster transform which may introduce some error)
    x_hat_top_k_g, x_hat_bot_k_g =  get_top_bot_k_vec(dct(x_g, norm='ortho'),k=k)    
    e_g = y_g - x_g
    eta_g = bot_l2_bp[i,1] = np.linalg.norm(x_hat_bot_k_g)
    
    
    y_b = m_data_y_bp[i,:,:,2].flatten()
    x_b= m_data_sub_bp[i,:,:,2].flatten()
    
    #Get actual top k and bottom k (we use the faster transform which may introduce some error)
    x_hat_top_k_b, x_hat_bot_k_b =  get_top_bot_k_vec(dct(x_b, norm='ortho'),k=k)    
    e_b = y_b- x_b
    eta_b = bot_l2_bp[i,2] = np.linalg.norm(x_hat_bot_k_b)


    args = [(y_r,eta_r), (y_g,eta_g), (y_b,eta_b)]
    p = Pool(3)
    approximates = p.map(approximate, args)
    p.terminate()

    x_hat_approx_r = approximates[0]
    x_hat_approx_g = approximates[1]
    x_hat_approx_b = approximates[2]
    
    
    delta_r = np.sqrt( (c*float(k)*t_values_bp[i,0])/float(n))
    mu_r = np.sqrt(1 + delta_r)/(1-delta_r)
    theta_r = (np.sqrt(k+t_values_bp[i,0])/(1 - delta_r))*np.sqrt((float(k)*c)/float(n))
    beta = np.sqrt((float(k)*c)/float(n))
    upper_r = ( (2*mu_r*np.sqrt(k+t_values_bp[i,0]))/(1-theta_r) )*(1 + (beta/(1-delta_r)) )*eta_r
    
        
    delta_g = np.sqrt( (c*float(k)*t_values_bp[i,1])/float(n))
    mu_g = np.sqrt(1 + delta_g)/(1-delta_g)
    theta_g = (np.sqrt(k+t_values_bp[i,1])/(1 - delta_g))*np.sqrt((float(k)*c)/float(n))
    upper_g = ( (2*mu_g*np.sqrt(k+t_values_bp[i,1]))/(1-theta_g) )*(1 + (beta/(1-delta_g)) )*eta_g
    
        
    delta_b = np.sqrt( (c*float(k)*t_values_bp[i,2])/float(n))
    mu_b = np.sqrt(1 + delta_b)/(1-delta_b)
    theta_b = (np.sqrt(k+t_values_bp[i,2])/(1 - delta_b))*np.sqrt((float(k)*c)/float(n))
    upper_b = ( (2*mu_b*np.sqrt(k+t_values_bp[i,2]))/(1-theta_b) )*(1 + (beta/(1-delta_b)) )*eta_b
    
    #Note the errors
    errors_l2_bp[i] = np.linalg.norm(x_hat_top_k_r.flatten()- x_hat_approx_r.flatten()) + \
                        np.linalg.norm(x_hat_top_k_g.flatten()- x_hat_approx_g.flatten()) + \
                         np.linalg.norm(x_hat_top_k_b.flatten()- x_hat_approx_b.flatten())
   
    #Calculate the difference from the upper bound
    diff_l2_bp[i] = (upper_r + upper_g + upper_b) - errors_l2_bp[i]

In [13]:
print(np.mean(t_values_bp), 
      np.mean(errors_l2_bp), 
      np.mean(diff_l2_bp))

4.032666666666667 20.105961380672948 866.0788459597803


In [14]:
mnist_tup_bp = (m_data_y_bp, m_data_sub_bp, t_values_bp,errors_l2_bp, diff_l2_bp )

import pickle
with open('data/cifar10_theory_socp_l0.pickle', 'wb') as f:
    pickle.dump(mnist_tup_bp, f)
